# Phase 1: Data Audit & Cleaning

This notebook documents the data validation, audit and transformation process for the sales analytics assignment, working with three source datasets: `Customer`, `Order`, and `Shipping`.

## Load Data

In [1]:
import pandas as pd

# Loading the datasets
customer_df = pd.read_csv('/Users/mitthooo/Downloads/Customer - atkoe-u250m.csv')  
order_df=pd.read_csv('/Users/mitthooo/Downloads/Order.csv') 
shipping_df=pd.read_json('/Users/mitthooo/Downloads/Shipping.json')


## Basic Data Validation & Exploration

In [7]:
#Customer_file

customer_df.info()
customer_df.isnull().sum()
customer_df.duplicated().sum()
customer_df['Age'].describe()
customer_df['Country'].value_counts()

#Order_file
order_df.info()
order_df.isnull().sum()
order_df.duplicated().sum()
order_df['Amount'].describe()
order_df['Item'].value_counts()
order_df['Customer_ID'].nunique()

#Shipping_file
shipping_df.info()
shipping_df.isnull().sum()
shipping_df.duplicated().sum()
shipping_df['Status'].value_counts()
shipping_df['Customer_ID'].nunique()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Customer_ID  250 non-null    int64 
 1   First        250 non-null    object
 2   Last         250 non-null    object
 3   Age          250 non-null    int64 
 4   Country      250 non-null    object
dtypes: int64(2), object(3)
memory usage: 9.9+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Order_ID     250 non-null    int64 
 1   Item         250 non-null    object
 2   Amount       250 non-null    int64 
 3   Customer_ID  250 non-null    int64 
dtypes: int64(3), object(1)
memory usage: 7.9+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------   

154

## Audit Function
We'll use a helper function to inspect column-level metadata such as data type, null values, unique values, and a few sample values.

In [8]:
def audit_dataframe(df, name):
    summary = {
        "Column": [],
        "Data Type": [],
        "Missing Values": [],
        "Unique Values": [],
        "Sample Values": [],
    }

    for col in df.columns:
        summary["Column"].append(col)
        summary["Data Type"].append(str(df[col].dtype))
        summary["Missing Values"].append(df[col].isnull().sum())
        summary["Unique Values"].append(df[col].nunique())
        summary["Sample Values"].append(df[col].dropna().unique()[:5])

    summary_df = pd.DataFrame(summary)
    summary_df.insert(0, "Dataset", name)
    return summary_df


## Run Audit on All Datasets

In [9]:
customer_audit = audit_dataframe(customer_df, "Customer")
order_audit = audit_dataframe(order_df, "Order")
shipping_audit = audit_dataframe(shipping_df, "Shipping")

# Combine all results
full_audit = pd.concat([customer_audit, order_audit, shipping_audit], ignore_index=True)
full_audit

,Dataset,Column,Data Type,Missing Values,Unique Values,Sample Values
0,Customer,Customer_ID,int64,0,250,"[1, 2, 3, 4, 5]"
1,Customer,First,object,0,171,"[Joseph, Gary, John, Eric, William]"
2,Customer,Last,object,0,189,"[Rice, Moore, Walker, Carter, Jackson]"
3,Customer,Age,int64,0,62,"[43, 71, 44, 38, 58]"
4,Customer,Country,object,0,3,"[USA, UK, UAE]"
5,Order,Order_ID,int64,0,250,"[1, 2, 3, 4, 5]"
6,Order,Item,object,0,8,"[Keyboard, Mouse, Monitor, Mousepad, Harddisk]"
7,Order,Amount,int64,0,9,"[400, 300, 12000, 250, 5000]"
8,Order,Customer_ID,int64,0,160,"[139, 250, 239, 153, 164]"
9,Shipping,Shipping_ID,int64,0,250,"[1, 2, 3, 4, 5]"


## Summary
We verified the datasets against key quality checks:

- Null values
- Duplicate entries
- Key column uniqueness
- Range and domain constraints
- Referential integrity between Customer_IDs across tables

All datasets passed these checks and are clean for modeling.

##  Referential Integrity (Joins)
Check if foreign keys match:

In [12]:

# Customers in orders but not in customers
missing_in_customers_from_orders = set(order_df['Customer_ID']) - set(customer_df['Customer_ID'])
print("Customers in orders but missing in customers table:", missing_in_customers_from_orders)

# Customers in shipping but not in customers
missing_in_customers_from_shipping = set(shipping_df['Customer_ID']) - set(customer_df['Customer_ID'])
print("Customers in shipping but missing in customers table:", missing_in_customers_from_shipping)



Customers in orders but missing in customers table: set()
Customers in shipping but missing in customers table: set()


### Next Step
We are now ready to move to **Phase 2: Domain Modeling**, where we'll create an ERD and structure the data to support the reporting requirements.